<a href="https://colab.research.google.com/github/DiffusionDalmation/pt_to_safetensors_converter_notebook/blob/main/pt_to_safetensors_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title  Mount Google Drive
from google.colab import drive
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - If you're not using a shared drive, leave this empty

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive 
else:
  mainpth="MyDrive"

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@title Install Required Dependencies
!pip install torch
!pip install safetensors

In [ ]:
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
file_path = "" #@param {type:"string"}pt
#@markdown - Copy and paste the path to an embedding, or a directory containing several embeddings
#@markdown - For example: /content/gdrive/MyDrive/myembedding.pt or /content/gdrive/MyDrive/my_directory
#@markdown - Embeddings must be in .pt format
embedding_info = True #@param {type:"boolean"}
#@markdown - Check this box to get additional information about the embedding you're converting

In [ ]:
#@title Define Converter Function
import os
import torch
from safetensors.torch import save_file

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def process_pt_files(path, verbose=True):
    if os.path.isdir(path):
        # Path is a directory, process all .pt files in the directory
        for file_name in os.listdir(path):
            if file_name.endswith('.pt'):
                process_file(os.path.join(path, file_name), verbose)
    elif os.path.isfile(path) and path.endswith('.pt'):
        # Path is a .pt file, process this file
        process_file(path, verbose)
    else:
        print(f"{path} is not a valid directory or .pt file.")

def process_file(file_path, verbose):
    # Load the PyTorch model
    model = torch.load(file_path, map_location=device)
    
    #Extract the embedding tensors
    model_tensors = model.get('string_to_param').get('*')
    s_model = {
          'emb_params': model_tensors
            }
    
    if verbose:
      print(file_path)
    # Print the requested training information, if it exists
      if ('sd_checkpoint_name' in model) and (model['sd_checkpoint_name'] is not None):
            print(f"Trained on {model['sd_checkpoint_name']}.")
      else:
            print("Checkpoint name not found in the model.")
        
      if ('step' in model) and (model['step'] is not None):
            print(f"Trained for {model['step']} steps.")
      else:
            print("Step not found in the model.")
      # Display the tensor's shape
      print(f"Dimensions of embedding tensor: {model_tensors.shape}")
      print()

    # Save the model with the new extension
    new_file_path = file_path.replace('.pt', '.safetensors')
    torch.save(s_model, new_file_path)


In [ ]:
#@title Convert the Embedding(s)
process_pt_files(file_path, verbose=embedding_info)

The converted safetensors embedding will be saved in the same directory as the original.